# Evaluate the entire processing of a set of ten protocols

In [1]:
import sys
sys.path.append('../front_end')

import pandas as pd
from sklearn.metrics import accuracy_score
import os
from util.protocol_master_processor import MasterProcessor
import re

2023-01-23 13:01:01.731991: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-23 13:01:07.007591: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-23 13:01:07.007660: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-23 13:01:07.486780: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-23 13:01:26.138475: W tensorflow/stream_executor/platform/de

In [2]:
df_testing = pd.read_excel("dsm_string_testing_v01.xlsx")

In [3]:
df_annotations = pd.read_csv("../data/ctgov/annotations/all_annotations.csv")

In [4]:
df_annotations.set_index("nct_id", inplace=True)

In [5]:
cwd = os.getcwd()

In [6]:
os.chdir('../front_end')

In [7]:
master_processor = MasterProcessor("models/condition_classifier.pkl.bz2",
                                   "models/phase_rf_classifier.pkl.bz2",
                                   "models/spacy-textcat-phase-04-model-best",
                                   "models/sap_classifier_document_level.pkl.bz2",
                                   "models/sap_classifier.pkl.bz2",
                                   "models/effect_estimate_classifier.pkl.bz2",
                                   "models/num_subjects_classifier.pkl.bz2",
                                   "models/subjects_classifier_document_level.pkl.bz2",
                                   "models/arms_classifier_document_level.pkl.bz2",
                                   "models/spacy-textcat-arms-21-model-best",
                                   "models/spacy-textcat-international-11-model-best",
                                   "models/spacy-textcat-country-16-model-best",
                                   "models/international_classifier.pkl.bz2",
                                   "models/country_ensemble_model.pkl.bz2",
                                   "models/simulation_classifier.pkl.bz2")

Initialising Phase Random Forest classifier models/phase_rf_classifier.pkl.bz2


/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/protocols/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/protocols/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/protocols/lib/python3.10/site-pa

Initialising SAP document level classifier models/sap_classifier_document_level.pkl.bz2
Initialising Num Arms classifier models/arms_classifier_document_level.pkl.bz2


/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/protocols/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/protocols/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/protocols/lib/python3.10/site-packages/sklear

Initialising int classifier models/international_classifier.pkl.bz2


/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/protocols/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator BernoulliNB from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
ctgov_n = []
ctgov_sap = []
ctgov_countries = []

for idx in range(len(df_testing)):
    nct = df_testing.Protocol.iloc[idx]
    n = None
    sap = None
    if nct in df_annotations.index:
        n = df_annotations.num_subjects[nct]
        sap = df_annotations.has_sap[nct]
        countries = df_annotations.country[nct]
    ctgov_n.append(n)
    ctgov_sap.append(sap)
    ctgov_countries.append(countries)
df_testing.insert(df_testing.columns.get_loc("Rev_N") + 1, "CTGov_N", ctgov_n)
df_testing.insert(df_testing.columns.get_loc("Rev_SAP") + 1, "CTGov_SAP", ctgov_sap)
df_testing.insert(df_testing.columns.get_loc("Rev_countries") + 1, "CTGov_countries", ctgov_countries)

In [9]:
file_to_page = {}
import json
texts = []
for idx in range(len(df_testing)):
    nct = df_testing.Protocol.iloc[idx]
    file_name = None
    if nct in df_annotations.index:
        file_name = df_annotations.file[nct]
    print (file_name)
    with open("../data/ctgov/json/" + file_name + ".json", "r", encoding="utf-8") as f:
        pages = json.load(f)
    texts.append(pages)
    file_to_page[file_name] = pages
df_testing["pages"] = texts

56_NCT00752856_Prot_SAP_000.pdf
21_NCT01896921_Prot_SAP_000.pdf
94_NCT01933594_Prot_001.pdf
04_NCT02217904_Prot_SAP_000.pdf
26_NCT02263326_Prot_SAP_ICF_000.pdf
61_NCT02707861_Prot_000.pdf
75_NCT02788175_Prot_SAP_001.pdf
47_NCT02946047_Prot_SAP_000.pdf
41_NCT03262441_Prot_SAP_000.pdf
99_NCT03351699_Prot_SAP_000.pdf
86_NCT02342886_Prot_000.pdf
28_NCT02153528_Prot_000.pdf
48_NCT02583048_Prot_001.pdf
76_NCT02193776_Prot_000.pdf
72_NCT02410772_Prot_SAP_000.pdf
84_NCT02114684_Prot_000.pdf


In [10]:
# import bz2, pickle as pkl
# with bz2.open("demo_data/demo_protocols.pkl.bz2", "wb") as f:
#     pkl.dump(file_to_page, f)

In [11]:
pred_cond = []
pred_phase = []
pred_arms = []
pred_sap = []
pred_subjects = []
pred_sim = []
pred_effect = []
pred_countries = []

for idx in range(len(df_testing)):
    tokenised_pages, condition_to_pages, phase_to_pages, sap_to_pages, \
               effect_estimate_to_pages, num_subjects_to_pages,\
    num_arms_to_pages, country_to_pages, simulation_to_pages = master_processor.process_protocol(df_testing["pages"].iloc[idx])
    pred_cond.append(condition_to_pages["prediction"])
    pred_phase.append(phase_to_pages["prediction"])
    pred_arms.append(num_arms_to_pages["prediction"])
    pred_sap.append(sap_to_pages["prediction"])
    pred_subjects.append(num_subjects_to_pages["prediction"])    
    pred_sim.append(simulation_to_pages["prediction"])    
    pred_effect.append(effect_estimate_to_pages["prediction"])        
    pred_countries.append(country_to_pages["prediction"])        

Splitting the document into words (tokens)...
There were 19078 words in the document.

Searching for a likely pathology...
This looks like an HIV trial.

Searching for a phase...
This looks like a Phase 2.0 trial.

Neural network thought it was a Phase 2.0 trial.

Searching for a statistical analysis plan...
It does not look like the protocol contains a statistical analysis plan.

Testing top pages for SAP with document level SAP Naive Bayes model to refine SAP prediction.

Document level Naive Bayes model found SAP score 1 with score 0.5583868265125077.

Searching for an effect estimate...


/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/protocols/lib/python3.10/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Identified probable effect estimate.

Naive Bayes arms prediction probabilities: {'1': 0.17866803880560597, '2': 0.5245156463204527, '3+': 0.29681631487394244}.

Spacy arms prediction probabilities: {'1': 0.008743556216359138, '2': 0.9820131659507751, '3+': 0.009243257343769073}.

Searching for a number of arms...
No explicit mention of arms found.

Running Random Forest classifier for number of subjects...
Searching for a number of subjects...
It looks like the trial has 50 participants.

Searching for the countries of investigation...
It looks like the trial takes place in 1 country: PE

Neural network found that trial country is likely to be USCA.

Neural network for is trial international? output: 0.

Naive Bayes model for is trial international? output: 0.

Ensemble model output: {"prediction": ["US", "CA"], "score": {"PE": 0.07, "XX": 0.09, "US": 0.77, "CA": 0.53}}

Searching for any mentions of simulation...
It does not look like the authors used simulation for sample size.

Spl

/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/protocols/lib/python3.10/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Identified probable effect estimate.

Naive Bayes arms prediction probabilities: {'1': 0.15118132734068362, '2': 0.42239348119637854, '3+': 0.42642519146293606}.

Spacy arms prediction probabilities: {'1': 8.177267890596696e-12, '2': 1.6423006617832803e-14, '3+': 1.0}.

Searching for a number of arms...
No explicit mention of arms found.

Running Random Forest classifier for number of subjects...
Searching for a number of subjects...
It looks like the trial has 12 participants.

Searching for the countries of investigation...
It looks like the trial takes place in 1 country: US

Neural network found that trial country is likely to be USCA.

Neural network for is trial international? output: 1.

Naive Bayes model for is trial international? output: 0.

Ensemble model output: {"prediction": ["US", "XX"], "score": {"US": 0.86, "XX": 0.51, "PR": 0.2525, "AU": 0.1}}

Searching for any mentions of simulation...
It does not look like the authors used simulation for sample size.

Splitting the

...

Identified probable effect estimate.

Naive Bayes arms prediction probabilities: {'1': 0.12485820690180549, '2': 0.5078040175363748, '3+': 0.36733777556181874}.

Spacy arms prediction probabilities: {'1': 1.773104355092267e-23, '2': 0.00025447257212363183, '3+': 0.9997455477714539}.

Searching for a number of arms...
No explicit mention of arms found.

Running Random Forest classifier for number of subjects...
Searching for a number of subjects...
It looks like the trial has 240 participants.

Searching for the countries of investigation...
It looks like the trial takes place in 1 country: ZA

Neural network found that trial country is likely to be USCA.

Neural network for is trial international? output: 1.

Naive Bayes model for is trial international? output: 1.

Ensemble model output: {"prediction": ["US", "XX", "ZA", "MX"], "score": {"ZA": 0.38, "XX": 0.46228882833787466, "US": 0.55, "MX": 0.1370166755840913}}

Searching for any mentions of simulation...
It does not look like the 

/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/protocols/lib/python3.10/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Identified probable effect estimate.

Naive Bayes arms prediction probabilities: {'1': 0.08751904288587753, '2': 0.6205856619163611, '3+': 0.29189529519775914}.

Spacy arms prediction probabilities: {'1': 0.9999592304229736, '2': 4.0807433833833784e-05, '3+': 2.2045772325729462e-12}.

Searching for a number of arms...
No explicit mention of arms found.

Running Random Forest classifier for number of subjects...
Searching for a number of subjects...
It looks like the trial has 90 participants.

Searching for the countries of investigation...
It looks like the trial takes place in 1 country: US

Neural network found that trial country is likely to be USCA.

Neural network for is trial international? output: 1.

Naive Bayes model for is trial international? output: 0.

Ensemble model output: {"prediction": ["US"], "score": {"US": 0.77, "XX": 0.23, "HK": 0.32, "CN": 0.18, "ZA": 0.33, "VN": 0.34, "FR": 0.32, "ES": 0.32, "ZW": 0.18, "KE": 0.11, "PE": 0.11, "UG": 0.11, "SG": 0.2, "BE": 0.13}}

/media/thomas/642d0db5-2c98-4156-b591-1a3572c5868c/anaconda3/envs/protocols/lib/python3.10/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


Identified probable effect estimate.

Naive Bayes arms prediction probabilities: {'1': 0.12629555371977186, '2': 0.593418011984293, '3+': 0.28028643429593475}.

Spacy arms prediction probabilities: {'1': 0.0014953858917579055, '2': 0.953068196773529, '3+': 0.04543643444776535}.

Searching for a number of arms...
No explicit mention of arms found.

Running Random Forest classifier for number of subjects...
Searching for a number of subjects...
It looks like the trial has 330 participants.

Searching for the countries of investigation...
It looks like the trial takes place in 1 country: ZA

Neural network found that trial country is likely to be USCA.

Neural network for is trial international? output: 0.

Naive Bayes model for is trial international? output: 0.

Ensemble model output: {"prediction": ["ZA", "US"], "score": {"ZA": 0.91, "XX": 0.0, "CO": 0.12, "UG": 0.07, "GE": 0.02, "TZ": 0.02, "KE": 0.04, "US": 0.69, "CH": 0.02, "BW": 0.04}}

Searching for any mentions of simulation...
T

In [12]:
df_testing.insert(df_testing.columns.get_loc("Indication") + 1, "AI_Indication_v2", pred_cond)
df_testing.insert(df_testing.columns.get_loc("AI_phase") + 1, "AI_phase_v2", pred_phase)
df_testing.insert(df_testing.columns.get_loc("Rev_arms") + 1, "AI_arms_v2", pred_arms)
df_testing.insert(df_testing.columns.get_loc("AI_SAP") + 1, "AI_SAP_v2", pred_sap)
df_testing.insert(df_testing.columns.get_loc("AI_N") + 1, "AI_N_v2", pred_subjects)
df_testing.insert(df_testing.columns.get_loc("AI_effect") + 1, "AI_effect_v2", pred_effect)
df_testing.insert(df_testing.columns.get_loc("AI_sim") + 1, "AI_sim_v2", pred_sim)
df_testing.insert(df_testing.columns.get_loc("Rev_countries") + 1, "AI_countries_v2", [",".join(sorted(set(c))) for c in pred_countries])


In [13]:
accuracies = {}

In [14]:
phase_map = {"1/2":"0.5"}
accuracies['AI_phase'] = accuracy_score(df_testing.Rev_phase.apply(str).apply(lambda x : phase_map.get(x, x)), df_testing.AI_phase.apply(str))

accuracies['AI_phase_v2'] = accuracy_score(df_testing.Rev_phase.apply(str), df_testing.AI_phase_v2.apply(str).apply(lambda x : re.sub(r'\.0', '', x)))

print (f"Phase accuracy v1: {accuracies['AI_phase']}")
print (f"Phase accuracy v2: {accuracies['AI_phase_v2']}")

Phase accuracy v1: 0.5
Phase accuracy v2: 0.6875


In [15]:
df_testing[["AI_effect", "Rev_effect", "AI_effect_v2"]]

,AI_effect,Rev_effect,AI_effect_v2
0,yes,yes Sec. 9.3,1
1,no,no,0
2,no,yes Sec. 9.4.2,1
3,no,yes Sec. 8.1ff,1
4,yes,yes Sec. 9.4,1
5,no,yes Sec. 9.8,0
6,no,yes Sec. 13,0
7,no,no,0
8,no,yes Sec. 8.4.1,1
9,no,yes Sec. 8.1ff,1


In [16]:
accuracies['AI_effect'] = accuracy_score(df_testing.Rev_effect.str.contains("yes").apply(int), df_testing.AI_effect.str.contains("yes").apply(int))
accuracies['AI_effect_v2'] = accuracy_score(df_testing.Rev_effect.str.contains("yes").apply(int), df_testing.AI_effect_v2)

print (f"Effect accuracy v1: {accuracies['AI_effect']}")
print (f"Effect accuracy v2: {accuracies['AI_effect_v2']}")

Effect accuracy v1: 0.5625
Effect accuracy v2: 0.875


In [17]:
clean_sap_gt = df_testing.Rev_SAP.apply(lambda x : int(float(re.sub(r'p.+', '', x)) > 1))

In [18]:
df_testing[["AI_SAP", "Rev_SAP", "AI_SAP_v2"]]

,AI_SAP,Rev_SAP,AI_SAP_v2
0,no,3pp in prot,1
1,no,0.5p in prot,1
2,no,9pp in prot,1
3,yes,4pp in prot,0
4,no,2pp in prot,1
5,no,3pp in prot,0
6,no,2pp in prot,1
7,no,2pp in prot,1
8,no,1p in prot,1
9,no,1p in prot,0


In [19]:
accuracies['AI_SAP'] = accuracy_score(clean_sap_gt, df_testing.AI_SAP.map({"yes":1,"no":0}))
accuracies['AI_SAP_v2'] = accuracy_score(clean_sap_gt, df_testing.AI_SAP_v2)

print (f"SAP accuracy v1: {accuracies['AI_SAP']}")
print (f"SAP accuracy v2: {accuracies['AI_SAP_v2']}")

SAP accuracy v1: 0.3125
SAP accuracy v2: 0.625


In [20]:
df_testing[["AI_N", "Rev_N", "AI_N_v2"]]

,AI_N,Rev_N,AI_N_v2
0,50,51,50
1,44,30,30
2,60,59,12
3,42,42,42
4,90,89,90
5,100,79,100
6,52,26,20
7,49,17,17
8,450,5,12
9,900,24,30


In [21]:
accuracies['AI_N'] = accuracy_score(df_testing.Rev_N, df_testing.AI_N)
accuracies['AI_N_v2'] = accuracy_score(df_testing.Rev_N, df_testing.AI_N_v2.apply(int))

print (f"Sample size accuracy v1: {accuracies['AI_N']}")
print (f"Sample size accuracy v2: {accuracies['AI_N_v2']}")

Sample size accuracy v1: 0.25
Sample size accuracy v2: 0.4375


In [22]:
df_testing[["AI_sim", "Rev_sim", "AI_sim_v2"]]

,AI_sim,Rev_sim,AI_sim_v2
0,no,no,0
1,no,no,0
2,no,no,0
3,no,no,0
4,no,no,0
5,no,no,0
6,no,no,0
7,no,no,0
8,no,no,0
9,no,no,0


In [23]:
accuracies['AI_arms_v2'] = accuracy_score(df_testing.Rev_arms, df_testing.AI_arms_v2)

print (f"Arms accuracy v2: {accuracies['AI_arms_v2']}")

Arms accuracy v2: 0.6875


In [24]:
df_testing[["Rev_countries", "AI_countries_v2"]]

,Rev_countries,AI_countries_v2
0,US,"CA,US"
1,US,US
2,US,"US,XX"
3,DE,"CA,US,XX"
4,US,US
5,"US,PR","TW,US,XX"
6,US,US
7,US,US
8,US,US
9,DE,"CA,HT,JE,US,XX"


In [25]:
exact_match = []
approximate_match = []
for i in range(len(df_testing)):
    countries_gt = set(df_testing.Rev_countries.iloc[i].split(","))
    countries_pred = set(df_testing.AI_countries_v2.iloc[i].split(","))
    exact_match.append(int(countries_gt == countries_pred))
    
    is_approximate_match = True
    for c in countries_gt:
        if c not in countries_pred and "XX" not in countries_pred:
            is_approximate_match = False
            
    for c in countries_pred:
        if c not in countries_gt:
            is_approximate_match = False
            
    approximate_match.append(int(is_approximate_match))
    
df_testing["is_country_exact_match"] = exact_match
df_testing["is_country_approximate_match"] = approximate_match

In [26]:
accuracies['AI_countries_v2'] = df_testing["is_country_exact_match"].mean()
accuracies['is_country_approximate_match'] = df_testing["is_country_approximate_match"].mean()

print (f"Countries accuracy v2: {accuracies['AI_countries_v2']}")
print (f"Countries accuracy v2 including approximate matches: {accuracies['is_country_approximate_match']}")

Countries accuracy v2: 0.3125
Countries accuracy v2 including approximate matches: 0.3125


In [27]:
df_subtotals = pd.DataFrame()
for idx, c in enumerate(df_testing.columns):
    last_row = ""
    if idx == 0:
        last_row = "ACCURACY"
    elif c in accuracies:
        last_row = accuracies[c]
    
    df_subtotals[c] = ["", last_row]

In [28]:
df_testing_with_subtotals = pd.concat([df_testing, df_subtotals]).drop(columns="pages")

In [29]:
df_testing_with_subtotals

,Protocol,Indication,AI_Indication_v2,Drug_or_biologic,Test_article,Pages,AI_phase,AI_phase_v2,Rev_phase,AI_SAP,...,Rev_sim,AI_risk,Rev_risk,Rev_arms,AI_arms_v2,Rev_countries,AI_countries_v2,CTGov_countries,is_country_exact_match,is_country_approximate_match
0,NCT00752856,HIV,HIV,drug,raltegravir_lopinavir_ritonavir,57,1,2.0,2,no,...,no,high,high,2,2,US,"CA,US",US,0,0
1,NCT01896921,HIV,HIV,drug,maraviroc_integrase_inhib,29,3,3.0,3,no,...,no,high,high,1,1,US,US,US,1,1
2,NCT01933594,HIV,HIV,drug,romidepsin,140,1,1.5,1/2,no,...,no,high,low,8,3,US,"US,XX",US,0,0
3,NCT02217904,HIV,HIV,drug,islatravir,114,1,1.0,1,yes,...,no,med,low,1,1,DE,"CA,US,XX",DE,0,0
4,NCT02263326,HIV,HIV,drug,dolutegravir_lamivudin,47,0,2.0,3,no,...,no,NaN,med,2,2,US,US,US,1,1
5,NCT02707861,HIV,HIV,biologic,ibalizumab,106,3,3.0,3,no,...,no,high,med,2,2,"US,PR","TW,US,XX","US,PR",0,0
6,NCT02788175,HIV,HIV,biologic,vedolizumab,55,3,1.0,1,no,...,no,high,med,1,1,US,US,US,1,1
7,NCT02946047,HIV,HIV,drug,ixazomib,83,1,1.5,1/2,no,...,no,high,high,4,1,US,US,US,1,1
8,NCT03262441,HIV,HIV,drug,mycophenolate_mofetil,39,2,2.0,2,no,...,no,high,high,1,1,US,US,US,1,1
9,NCT03351699,HIV,HIV,drug,MK-4250,87,1,1.0,1,no,...,no,high,med,5,3,DE,"CA,HT,JE,US,XX",DE,0,0


In [30]:
df_testing_with_subtotals.to_excel(cwd + "/README.xlsx", index=False)

In [31]:
df_testing_with_subtotals.to_markdown(cwd + "/README.md", index=False)